# Structured Planning Agent

A key pattern in agents is the ability to plan. ReAct for example, uses a structured approach to decompose an input into a set of function calls and thoughts, in order to reason about a final response.

However, breaking down the initial input/task into several sub-tasks can make the ReAct loop (or other reasoning loops) easier to execute.

The `StructuredPlanningAgnet` in LlamaIndex wraps any agent worker (ReAct, Function Calling, Chain-of-Abstraction, etc.) and decomposes an initial input into several sub-tasks. Each sub-task is represented by an input, expected outcome, and any dependendant sub-tasks that should be completed first.

This notebook walks through both the high-level and low-level usage of this agent.

**NOTE:** This agent leverages both structured outputs and agentic reasoning. Because of this, we would recommend a capable LLM (OpenAI, Anthropic, etc.), and open-source LLMs may struggle to plan without prompt engineering or fine-tuning.

## Setup

In order to create plans, we need a set of tools to create plans on top of. Here, we use some classic 10k examples.

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Use ollama in JSON mode
Settings.llm = OpenAI(
    model="gpt-4-turbo",
    temperature=0.1,
)
Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool

# Load documents, create tools
lyft_documents = SimpleDirectoryReader(
    input_files=["./data/10k/lyft_2021.pdf"]
).load_data()
uber_documents = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()

lyft_index = VectorStoreIndex.from_documents(lyft_documents)
uber_index = VectorStoreIndex.from_documents(uber_documents)

lyft_tool = QueryEngineTool.from_defaults(
    lyft_index.as_query_engine(),
    name="lyft_2021",
    description="Useful for asking questions about Lyft's 2021 10-K filling.",
)

uber_tool = QueryEngineTool.from_defaults(
    uber_index.as_query_engine(),
    name="uber_2021",
    description="Useful for asking questions about Uber's 2021 10-K filling.",
)

## High Level API

In this section, we cover the high-level API for creating with and chatting with a structured planning agent.

### Create the Agent

In [ ]:
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)

# create the function calling worker for reasoning
worker = FunctionCallingAgentWorker.from_tools(
    [lyft_tool, uber_tool], verbose=True
)

# wrap the worker in the top-level planner
agent = StructuredPlannerAgent(
    worker, tools=[lyft_tool, uber_tool], verbose=True
)

### Give the agent a complex task

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
response = agent.chat(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings."
)

=== Initial plan ===
Extract Lyft Risk Factors:
Summarize the key risk factors from Lyft's 2021 10-K filing. -> A summary of the key risk factors for Lyft as outlined in their 2021 10-K filing.
deps: []


Extract Uber Risk Factors:
Summarize the key risk factors from Uber's 2021 10-K filing. -> A summary of the key risk factors for Uber as outlined in their 2021 10-K filing.
deps: []


Combine Risk Factor Summaries:
Combine the summaries of key risk factors for Lyft and Uber from their 2021 10-K filings. -> A comprehensive summary of the key risk factors for both Lyft and Uber as outlined in their respective 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


> Running step 99c90044-9092-4e1a-828d-0a344fa1534f. Step input: Summarize the key risk factors from Lyft's 2021 10-K filing.
Added user message to memory: Summarize the key risk factors from Lyft's 2021 10-K filing.
> Running step a64c6c00-27f8-4e0e-ab6c-60e30bdbeba9. Step input: Summarize the ke

In [ ]:
print(str(response))

assistant: The 2021 10-K filings for Lyft and Uber reveal several key risk factors that both companies face, along with some unique challenges specific to each:

**Common Risk Factors:**
1. **Economic and Market Conditions**: Both companies are impacted by broader economic factors such as the COVID-19 pandemic, natural disasters, and economic downturns, which can affect demand and operational stability.
2. **Operational Challenges**: Challenges in achieving or maintaining profitability, intense competition in the ridesharing industry, and the unpredictability of financial results are common concerns.
3. **Technology and Cybersecurity**: Both companies emphasize the importance of managing security or privacy breaches and technological vulnerabilities, with Uber specifically noting risks from cyberattacks such as malware and phishing.
4. **Dependence on Third Parties**: Both rely on third-party providers for critical services, such as Amazon Web Services for Lyft and various service prov

## Low-level API [Advanced] 

In this section, we use the same agent, but expose the lower-level steps that are happening under the hood.

This is useful for when you want to expose the underlying plan, tasks, etc. to a human to modify them on the fly, or for debugging and running things step-by-step.

### Create the Agent

In [ ]:
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)

# create the react worker for reasoning
worker = FunctionCallingAgentWorker.from_tools(
    [lyft_tool, uber_tool], verbose=True
)

# wrap the worker in the top-level planner
agent = StructuredPlannerAgent(
    worker, tools=[lyft_tool, uber_tool], verbose=True
)

### Create the initial tasks and plan

In [ ]:
plan_id = agent.create_tasks(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings."
)

=== Initial plan ===
Extract Lyft Risk Factors:
What are the key risk factors listed in Lyft's 2021 10-K filing? -> A summary of key risk factors for Lyft from its 2021 10-K filing.
deps: []


Extract Uber Risk Factors:
What are the key risk factors listed in Uber's 2021 10-K filing? -> A summary of key risk factors for Uber from its 2021 10-K filing.
deps: []


Summarize Risk Factors:
Combine the summaries of key risk factors for Lyft and Uber from their 2021 10-K filings into a comprehensive overview. -> A comprehensive summary of the key risk factors for both Lyft and Uber based on their 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']




### Inspect the initial tasks and plan

In [ ]:
plan = agent.state.plan_dict[plan_id]

for sub_task in plan.sub_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Extract Lyft Risk Factors =====
Expected output:  A summary of key risk factors for Lyft from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Extract Uber Risk Factors =====
Expected output:  A summary of key risk factors for Uber from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Summarize Risk Factors =====
Expected output:  A comprehensive summary of the key risk factors for both Lyft and Uber based on their 2021 10-K filings.
Dependencies:  ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


### Execute the first set of tasks

Here, we execute the first set of tasks with their dependencies met.

In [ ]:
next_tasks = agent.state.get_next_sub_tasks(plan_id)

for sub_task in next_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

completed_tasks = []
for sub_task in next_tasks:
    response = agent.run_task(sub_task.name)
    completed_tasks.append((sub_task, response))
    agent.state.add_completed_sub_task(plan_id, sub_task)

===== Sub Task Extract Lyft Risk Factors =====
Expected output:  A summary of key risk factors for Lyft from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Extract Uber Risk Factors =====
Expected output:  A summary of key risk factors for Uber from its 2021 10-K filing.
Dependencies:  []
> Running step 45e01153-b625-4d90-869c-eb87da222c1b. Step input: What are the key risk factors listed in Lyft's 2021 10-K filing?
Added user message to memory: What are the key risk factors listed in Lyft's 2021 10-K filing?
=== Calling Function ===
Calling function: lyft_2021 with args: {"input": "What are the key risk factors listed in the 2021 10-K filing?"}
=== Function Output ===
The key risk factors listed in the 2021 10-K filing include:

1. General economic factors such as the impact of the COVID-19 pandemic, natural disasters, economic downturns, public health crises, and political crises.
2. Operational factors including the company's limited operating history, competition in the ind

If we wanted to, we could even execute each task in a step-wise fashion. It would look something like this:

```python
# Step-wise execution per task

for sub_task in next_tasks:
    # get the task from the state 
    task = agent.state.get_task(sub_task.name)

    # run intial resoning step
    step_output = agent.run_step(task.task_id)

    # loop until the last step is reached
    while not step_output.is_last:
        step_output = agent.run_step(task.task_id)
    
    # finalize the response and commit to memory
    agent.finalize_response(task.task_id, step_output=step_output)
```

### Check if we are done

If there are no remaining tasks, then we can stop. Otherwise, we can refine the current plan and continue

In [ ]:
next_tasks = agent.state.get_next_sub_tasks(plan_id)
print(len(next_tasks))

1


In [ ]:
for sub_task in next_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Summarize Risk Factors =====
Expected output:  A comprehensive summary of the key risk factors for both Lyft and Uber based on their 2021 10-K filings.
Dependencies:  ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


### Refine the plan

Since we have tasks remaining, lets refine our plan to make sure we are on track.

In [ ]:
# refine the plan
agent.refine_plan(
    plan_id,
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings.",
    completed_tasks,
)

=== Refined plan ===
Summarize Risk Factors:
Combine the summaries of key risk factors for Lyft and Uber from their 2021 10-K filings into a comprehensive overview. -> A comprehensive summary of the key risk factors for both Lyft and Uber based on their 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']




In [ ]:
plan = agent.state.plan_dict[plan_id]

for sub_task in plan.sub_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Summarize Risk Factors =====
Expected output:  A comprehensive summary of the key risk factors for both Lyft and Uber based on their 2021 10-K filings.
Dependencies:  ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


### Loop until done

With our plan refined, we can repeat this process until we have no more tasks to run.

In [ ]:
import asyncio

while True:
    # are we done?
    next_tasks = agent.state.get_next_sub_tasks(plan_id)
    if len(next_tasks) == 0:
        break

    # run concurrently for better performance
    responses = await asyncio.gather(
        *[agent.arun_task(sub_task.name) for sub_task in next_tasks]
    )
    for sub_task, response in zip(next_tasks, responses):
        completed_tasks.append((sub_task, response))
        agent.state.add_completed_sub_task(plan_id, sub_task)

    # are we done now?
    # LLMs have a tendency to add more tasks, so we end if there are no more tasks
    next_tasks = agent.state.get_next_sub_tasks(plan_id)
    if len(next_tasks) == 0:
        break

    # refine the plan
    await agent.arefine_plan(
        plan_id,
        "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings.",
        completed_tasks,
    )

> Running step e59c6ac0-8303-4f8f-acbc-4d9670362f1b. Step input: Combine the summaries of key risk factors for Lyft and Uber from their 2021 10-K filings into a comprehensive overview.
Added user message to memory: Combine the summaries of key risk factors for Lyft and Uber from their 2021 10-K filings into a comprehensive overview.
=== LLM Response ===
Here is a comprehensive overview of the key risk factors for both Lyft and Uber from their 2021 10-K filings:

### Common Risk Factors
1. **COVID-19 Pandemic**: Both companies highlight the adverse effects of the pandemic on their operations and overall business environment.
2. **Driver Classification**: Risk associated with the potential reclassification of drivers as employees rather than independent contractors, impacting business models and cost structures.
3. **Intense Competition**: Both face intense competition in the mobility and delivery sectors, with challenges from well-capitalized competitors, low barriers to entry, and the 

By the end, we should have a single response, which is our final response

In [ ]:
print(str(responses[-1]))

assistant: Here is a comprehensive overview of the key risk factors for both Lyft and Uber from their 2021 10-K filings:

### Common Risk Factors
1. **COVID-19 Pandemic**: Both companies highlight the adverse effects of the pandemic on their operations and overall business environment.
2. **Driver Classification**: Risk associated with the potential reclassification of drivers as employees rather than independent contractors, impacting business models and cost structures.
3. **Intense Competition**: Both face intense competition in the mobility and delivery sectors, with challenges from well-capitalized competitors, low barriers to entry, and the necessity to offer competitive pricing and incentives.
4. **Brand and Reputation**: Importance of maintaining a strong brand and reputation, especially given past negative publicity and the potential for illegal or improper user activities on their platforms.
5. **Financial Sustainability**: Challenges related to achieving or maintaining profi

## Changing Prompts

The `StructuredPlanningAgent` has two key prompts:
1. The initial planning prompt
2. The plan refinement prompt

Below, we show how to configure these prompts, using the defaults as an example.

In [ ]:
DEFAULT_INITIAL_PLAN_PROMPT = """\
Think step-by-step. Given a task and a set of tools, create a comprehesive, end-to-end plan to accomplish the task.
Keep in mind not every task needs to be decomposed into multiple sub-tasks if it is simple enough.
The plan should end with a sub-task that satisfies the overall task.

The tools available are:
{tools_str}

Overall Task: {task}
"""

DEFAULT_PLAN_REFINE_PROMPT = """\
Think step-by-step. Given an overall task, a set of tools, and completed sub-tasks, update (if needed) the remaining sub-tasks so that the overall task can still be completed.
The plan should end with a sub-task that satisfies the overall task.
If the remaining sub-tasks are sufficient, you can skip this step.

The tools available are:
{tools_str}

Overall Task:
{task}

Completed Sub-Tasks + Outputs:
{completed_outputs}

Remaining Sub-Tasks:
{remaining_sub_tasks}
"""

In [ ]:
agent = StructuredPlannerAgent(
    worker,
    tools=[lyft_tool, uber_tool],
    initial_plan_prompt=DEFAULT_INITIAL_PLAN_PROMPT,
    plan_refine_prompt=DEFAULT_PLAN_REFINE_PROMPT,
    verbose=True,
)